# 출력층 설계 (Output Layer)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F


def softmax(z):
    exp_z = np.exp(z)
    return exp_z / np.sum(exp_z)

x1 = np.array([1000, 1001, 1002]) # [nan, nan, nan] 왜 이런 결과가 나오나? 
# overflow 발생 : 지수 함수의 결과가 너무 크게 나옴

def stable_softmax(z): # 안정화된 softmax 함수, 예시에서는 1002가 제일 큰 수이므로 1002를 빼줌, np.array([-2, -1, 0]) → 오버플로우 발생하지 않음
    exp_z = np.exp(z - np.max(z))
    return exp_z / np.sum(exp_z)

x = np.array([10, 11, 12])
print(softmax(x))
print(sum(softmax(x)))
print(stable_softmax(x1))       # 오버플로우 발생하지 않음, softmax는 비율을 계산하는 것이므로 max(z)값을 똑같이 빼줘도 비율은 같음
print(sum(stable_softmax(x1)))


[0.09003057 0.24472847 0.66524096]
1.0
[0.09003057 0.24472847 0.66524096]
0.9999999999999999


In [9]:
np.exp(1000) # → 계산할 수 없는 범위가 된다. → 오버플로우 발생
# overflow encountered in exp  np.exp(1000)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_20040\3184034056.py:1: RuntimeWarning: overflow encountered in exp
  np.exp(1000) # → 계산할 수 없는 범위가 된다. → 오버플로우 발생


np.float64(inf)

In [2]:
def cross_entropy(y_pred, y_true):
    return -np.sum(y_true * np.log(y_pred))

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def relu(z):
    return np.maximum(0, z)

def leaky_relu(z, alpha=0.01):
    return np.maximum(alpha * z, z)

def tanh(z):
    return np.tanh(z)

def elu(z, alpha=1.0):
    return np.where(z > 0, z, alpha * (np.exp(z) - 1))

- pytorch 라이브러리 함수 사용

    딥러닝 프레임워크 → 딥러닝에 사용될 여러 함수를 제공하고 있다.

In [19]:
import torch
import torch.nn.functional as F

x = torch.tensor([1000, 1001, 1002], dtype=torch.float32)
softmax = nn.Softmax(dim=0)
print(softmax(x))
F.softmax(x, dim=0)

tensor([0.0900, 0.2447, 0.6652])


tensor([0.0900, 0.2447, 0.6652])

In [ ]:
x = torch.tensor([1000, 1001, 1002], dtype=torch.float32)

softmax_output = F.softmax(x, dim=0)    # 결과값이 stable_softmax 함수와 같음 
                                        # → 내부적으로 안정화된 softmax 함수를 사용하고 있음 → Overflow 발생하지 않음
print(softmax_output)

tensor([0.0900, 0.2447, 0.6652])


In [ ]:
x = np.array([1000, 1001, 1002])    → 반드시 tensor 형태로 변환해서 입력값을 만들어줘야한다.

softmax_output = F.softmax(x, dim=0)
print(softmax_output)

'numpy.ndarray' object has no attribute 'softmax' → numpy 배열에는 softmax 함수가 없음

AttributeError: 'numpy.ndarray' object has no attribute 'softmax'

In [ ]:
x = torch.tensor([1000, 1001, 1002])    → "softmax_lastdim_kernel_impl" not implemented for 'Long'
# 반드시 float32 형태로 변환해서 입력값을 만들어줘야한다. 데이터의 타입을 맞춰줘야한다.
# 우리는 softmax 함수를 사용할 것이기 때문에 long (int64) 타입이 아닌 float32 타입으로 변환해야한다.

softmax_output = F.softmax(x, dim=0)    # 결과값이 stable_softmax 함수와 같음 
                                        # → 내부적으로 안정화된 softmax 함수를 사용하고 있음 → Overflow 발생하지 않음
print(softmax_output)

NotImplementedError: "softmax_lastdim_kernel_impl" not implemented for 'Long'

In [ ]:
import torch
import torch.nn.functional as F

x = torch.tensor([1000, 1001, 1002], dtype=torch.float32)
softmax = nn.Softmax(dim=1) → dim 값이 1이면 차원 범위 오류 발생, dim 값은 0으로 설정정
Dimension out of range (expected to be in range of [-1, 0], but got 1)
print(softmax(x))
F.softmax(x, dim=0)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
import torch
import torch.nn.functional as F

x = torch.tensor([1000, 1001, 1002], dtype=torch.float32)

sigmoid_output = torch.sigmoid(x)
print(sigmoid_output)
print(F.sigmoid(x))

# → 결과값이 같음, 입력값이 모두 양수이므로 결과값이 모두 0.5 이상이 나옴

tensor([1., 1., 1.])
tensor([1., 1., 1.])


### 손실 함수와 연계

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
머신러닝에서는 sklearn 라이브러리를 사용하여 모델을 정의하고 학습을 진행했음
BaseEstimator 클래스를 상속받아 모델을 정의하고 학습을 진행했음.
그리고 fit, predict, score 메서드를 사용하여 모델을 학습하고 예측하고 평가를 진행했음.

딥러닝에서는 nn.Module 클래스를 상속받아 모델을 정의하고 학습을 진행.
forward 메서드를 사용하여 모델을 정의하고 학습을 진행.
backward 메서드를 사용하여 손실 함수를 계산하고 역전파를 진행.

손실 함수를 사용하여 모델을 학습하고 예측하고 평가를 진행. → 손실값을 갖고 모델을 학습하고 예측하고 평가를 진행.

In [34]:
# 간단한 다중 클래스 분류 모델 정의
class SimpleMultiClassModel(nn.Module):
    def __init__(self):                                            # → 모델의 구조를 정의하는 함수
        super(SimpleMultiClassModel, self).__init__()
        self.fc = nn.Linear(5, 3)    # fc : fully connected layer, 5개의 입력값을 3개의 출력값으로 변환
        # 5개의 입력값 : 4개의 특성 + 1개의 바이어스
        # 3개의 출력값 : 3개의 클래스
        # 입력층에서 출력층으로 가는 선형 변환 과정, 모두 연결되어 있음 → 모든 입력값이 출력값에 영향을 줌
        # nn.Linear() : 선형 변환 과정을 정의하는 함수 → 이 클래스는 선형 계산을 해주는 층이 존재하는 클래스

    # 전체 과정을 정의하는 함수
    def forward(self, x):                                          # → 순전파 과정을 정의하는 함수
        return self.fc(x)

    # # 손실 함수를 정의하는 함수                                    # → 본래는 손실함수를 만들어주는 것이 정석이나 지금 예제에서는 빼고 진행행
    # def loss_fn(self, y_pred, y_true):
    #     return F.cross_entropy(y_pred, y_true)

model = SimpleMultiClassModel()                                    # 우리가 만든 SimpleMultiClassModel 클래스를 사용하여 모델을 정의
criterion = nn.CrossEntropyLoss()                                  # → 손실 함수를 정의하는 함수
optimizer = optim.Adam(model.parameters(), lr=0.01)                # → 최적화 Adam 알고리즘 최적화 함수를 사용
# optimizer = optim.SGD(model.parameters(), lr=0.01)
# model.parameters() : 모델의 모든 파라미터를 가져오는 함수, → 가중치와 바이어스를 가져옴
# lr : 학습률, → 학습률은 0.01로 설정, 경사하강법을 통해서 가중치와 바이어스를 업데이트 함


In [43]:
# 데이터 생성
inputs = torch.randn(4, 5)                                        # Sameple 4개의 데이터, 5개의 특성
labels = torch.tensor([0, 2, 1, 0])                               # Sample 4개의 label 데이터, 3개의 클래스

# 모델 학습
for _ in range(10):                          # 학습을 임의로 10번 반복
    preds = model(inputs)                    # 순전파 과정    → 모델에 입력값을 넣어서 예측(출력)값을 얻음
    loss = criterion(preds, labels)          # 손실 함수 계산 → 예측값과 실제값을 비교하여 손실값을 계산
    print(loss.item())

    optimizer.zero_grad()                    # 기울기 초기화  → 기울기를 0으로 초기화, 이전 단계에서 계산된 기울기를 0으로 초기화
    loss.backward()                          # 역전파 과정    → 손실 함수를 미분하여 기울기를 계산, 손실에 대한 역전파 수행 (파라미터에 대한 기울기 계산)
    optimizer.step()                         # 가중치와 바이어스 업데이트 → 기울기를 사용하여 가중치와 바이어스를 업데이트
                                             # 계산된 기울기를 사용하여 옵티마이저가 모델의 파라미터 업데이트

0.8790372610092163
0.8682606220245361
0.8504062294960022
0.8268001079559326
0.7986413240432739
0.7670071125030518
0.7328615188598633
0.6970622539520264
0.6603679060935974
0.6234432458877563


In [ ]:
# 모델 학습
optimizer.zero_grad()
outputs = model(inputs)
loss = criterion(outputs, outputs)
loss.backward()
optimizer.step()